<a href="https://colab.research.google.com/github/anhle/tensorFlow2.x/blob/master/CNN_on_MNIST_handwriten_digit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a convolutional neural network with TensorFlow v2
We build CNN using a low-level approach to better understand all mechanics behind building convolutional neural networks and the training process.

CNN Overview

![alt text](https://camo.githubusercontent.com/afc8cecd033ab300799ceb2bf3b593efa3bda2b7/687474703a2f2f706572736f6e616c2e69652e6375686b2e6564752e686b2f7e63636c6f792f70726f6a6563745f7461726765745f636f64652f696d616765732f666967332e706e67)

MNIST Dataset Overview
This example is using MNIST handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 255.

In this example, each image will be converted to float32 and normalized to [0, 1].

MNIST Dataset

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Import TensorFlow
!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf
import tensorflow_datasets as tfds
from   tensorflow.python.ops import summary_ops_v2
from   tensorflow import keras
from tensorflow.keras import Model, datasets, layers,optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D,MaxPool2D,Dropout

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import  time

print(tf.__version__)

Let's start by loading MNIST from keras.datasets and preprocessing to get rows of normalized 784-dimensional vectors.



In [0]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255.0, x_test / 255.0

# Use tf.data API to shuffle and batch data.
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Build the tf.keras model using the Keras model subclassing API:



In [0]:
class MyConvNet(Model):
    # Set layers.
    def __init__(self):
        super(MyConvNet, self).__init__()
        # Convolution Layer with 32 filters and a kernel size of 5.
        self.conv1 = Conv2D(32, kernel_size=5, activation='relu')
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2. 
        self.maxpool1 = MaxPool2D(2, strides=2)

        # Convolution Layer with 64 filters and a kernel size of 3.
        self.conv2 = Conv2D(64, kernel_size=3, activation='relu')
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2. 
        self.maxpool2 = MaxPool2D(2, strides=2)

        # Flatten the data to a 1-D vector for the fully connected layer.
        self.flatten = Flatten()

        # Fully connected layer.
        self.fc1 = Dense(1024)
        # Apply Dropout (if is_training is False, dropout is not applied).
        self.dropout = Dropout(rate=0.5)

        # Output layer, class prediction (0-9 digits)
        self.out = Dense(10)

    # Set forward pass.
    def call(self, x, is_training=False):
        x = tf.reshape(x, [-1, 28, 28, 1])
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x, training=is_training)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

# Build neural network model.
conv_net = MyConvNet()

Choose an optimizer and loss function for training:

In [0]:
# Training parameters.
learning_rate = 0.001
training_steps = 200
batch_size = 128
display_step = 10

# Cross-Entropy Loss.
# Note that this will apply 'softmax' to the logits.
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [0]:
# Optimization process. 
@tf.function
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        # Forward pass.
        pred = conv_net(x, is_training=True)
        # Compute loss.
        loss = cross_entropy_loss(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = conv_net.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [0]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_ds.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = conv_net(batch_x)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))


In [0]:
# Test model on validation set.
pred = conv_net(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

In [0]:
# Predict 5 images from validation set.
n_images = 5
test_images = x_test[:n_images]
predictions = conv_net(test_images)

# Display image and model prediction.
for i in range(n_images):
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')
    plt.show()
    print("Model prediction: %i" % np.argmax(predictions.numpy()[i]))